In [1]:
import numpy as np

In [18]:
# Model parameters
median = 1.75/1000
γ = 0.024
τ = median * γ
δ = 0.01
η = 0.032
ξ_m = 0.00128

μ_2 = 1.
σ_2 = 1.2
ρ = 0.21*2/(σ_2**2*μ_2) # Match moments, using 100 year's std

Equation:

$$
0 = \max_{e} \min_{h_2} - \delta \phi(r, z_2) + \delta \eta \log e - \tau z_2 e - \frac{\partial \phi}{\partial r}(r, z_2) e + \xi_m\frac{(h_2)^2}{2} + \left[\frac{\partial \phi}{\partial z_2}(r, z_2)\right]\left[-\rho (z_2 -\mu_2) + \sqrt{z_2} \sigma_2 h_2\right] + \left[\frac{\partial^2 \phi}{\partial(z_2)^2}(r, z_2)\right]\left(\frac{z_2|\sigma_2|^2}{2}\right)
$$

FOC of $h_2$ gives : 

$$
h_2^* = -\frac{\left[\frac{\partial \phi}{\partial z_2}(r, z_2)\right]\sqrt{z_2}\sigma_2}{\xi_m}
$$

FOC of $e$ gives :

$$
e^* = \frac{\delta \eta}{\tau z_2 + \frac{\partial \phi}{\partial r}(r, z_2)}
$$

Substituting terms we have:

$$
0 = - \delta \phi(r, z_2) + \delta \eta \log e^* - \delta\eta - \frac{\xi_m}{2}\left[\frac{\partial \phi}{\partial z_2}(r, z_2)\right]^2z_2|\sigma_2|^2 + \left[\frac{\partial \phi}{\partial z_2}(r, z_2)\right]\left[-\rho (z_2 -\mu_2)\right] + \left[\frac{\partial^2 \phi}{\partial(z_2)^2}(r, z_2)\right]\left(\frac{z_2|\sigma_2|^2}{2}\right)
$$